In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity  
from nltk.corpus import stopwords
import numpy as np
import numpy.linalg as LA
import pandas as pd
from textblob import TextBlob
from nltk import word_tokenize

mylist = list()
InputSheet = "NewCodes2016Formatted.csv"
MasterSheet = "Same_SIM_Master_List.csv"
resultFrame = pd.DataFrame(columns=('NewCode','NewCodeDesc','SameSimCode', 'CPTDesc','Accuracy','Rank','Comments'),index=range(30000))
rownum = -1
NewCodeNum = -1

def create_tokenizer_score(new_series, train_series, tokenizer):
    """
    return the tf idf score of each possible pairs of documents
    Args:
        new_series (pd.Series): new data (To compare against train data)
        train_series (pd.Series): train data (To fit the tf-idf transformer)
    Returns:
        pd.DataFrame
    """
    train_tfidf = tokenizer.fit_transform(train_series)
    
    new_tfidf = tokenizer.transform(new_series)

    X = pd.DataFrame(cosine_similarity(new_tfidf, train_tfidf), columns=train_series.index)
    X['ix_new'] = new_series.index
    score = pd.melt(
        X,
        id_vars='ix_new',
        var_name='ix_train',
        value_name='score'
    )
    return score


def Get_MasterCodeDesc():
    df = pd.read_csv(MasterSheet)
    sMasterCodeDesc = df['CPTLongDesc'].tolist()
    return sMasterCodeDesc;

def GetNewCPT():
    NewCodeFrame = pd.read_csv(InputSheet,encoding ='latin1')
    sNewCode = NewCodeFrame['HCPCS'].tolist()
    return sNewCode;

def GetNewCPTDesc():
    NewCodeFrame = pd.read_csv(InputSheet,encoding ='latin1')
    sNewCodeDesc = NewCodeFrame['LongDesc'].tolist()
    return sNewCodeDesc;

def GetSameSIMData():
    SameSimDf = pd.read_csv(InputSheet,encoding ='latin1')
    sNewCodeDesc = SameSimDf['SimCode'].tolist()
    return sNewCodeDesc;
   
def textblob_tokenizer(str_input):
    blob = TextBlob(str_input.lower())
    tokens = blob.words
    words = [token.stem() for token in tokens]
    return words

def GetRowNum(): 
    try:
        rownum = resultFrame.loc[(resultFrame.Accuracy == '0003T') & (resultFrame.Rank == 'CERVICOGRAPY')].index[0]
            #print(rownum)
        resultFrame.loc[rownum,'Comments'] = "Test"
    except:
        "No record found"

def GetMasterCPTCode(Val,NewCode,NewCodeDesc,Accuracy,RankNo,Comment,x):
    df = pd.read_csv(MasterSheet)    
    GetCPTValue = df.iloc[Val,0]
    GetCPTDesc = df.iloc[Val,1]    
    mylist.append(GetCPTValue)
    mylist.append(GetCPTDesc)
    resultFrame.loc[x].NewCode = NewCode
    resultFrame.loc[x].SameSimCode = GetCPTValue
    resultFrame.loc[x].CPTDesc = GetCPTDesc
    resultFrame.loc[x].NewCodeDesc = NewCodeDesc
    resultFrame.loc[x].Accuracy = Accuracy
    resultFrame.loc[x].Rank = RankNo
    resultFrame.loc[x].Comments = Comment
    return mylist;

#train_CPT_Set = pd.Series(GetMasterCPT())
train_set = pd.Series(Get_MasterCodeDesc())
NewCodeDescSet = pd.Series(GetNewCPTDesc())
NewCodeSet = pd.Series(GetNewCPT())
SameSimSet = pd.Series(GetSameSIMData())
for i in NewCodeDescSet:
    NewCodeNum = NewCodeNum+1
    test_set = pd.Series(i)   
    try:
        rowFound = resultFrame.loc[(resultFrame.NewCode == NewCodeSet[NewCodeNum])].index[0]  
        try:        
            rowNumSameSimFound = resultFrame.loc[(resultFrame.NewCode == NewCodeSet[NewCodeNum]) & (resultFrame.SameSimCode == SameSimSet[NewCodeNum])].index[0]
            resultFrame.loc[rowNumSameSimFound,'Comments'] = "Present Both in MD and Autoproposal"
        except:
            try:
                rownum = rownum+1
                resultFrame.loc[rownum].NewCode = NewCodeSet[NewCodeNum]
                resultFrame.loc[rownum].SameSimCode = SameSimSet[NewCodeNum]
                resultFrame.loc[rownum].Comments = "Present only by MD"
            except:
                "Nothing"
    except:
        print(NewCodeSet[NewCodeNum])
        #print(i)
        test_set2 = word_tokenize(i)
        #tokenizer = TfidfVectorizer(stop_words = 'english')# initiate here your own tokenizer (TfidfVectorizer, CountVectorizer, with stopwords...)
        tokenizer = TfidfVectorizer()
        score = create_tokenizer_score(train_series=train_set, new_series=test_set, tokenizer=tokenizer)
        #score
        pd.set_option("display.max_rows", 11)
        sortedscore = score.sort_values(by=['score'], ascending=False)
        sortedscore.head(11)
        ranknum = 0
        found = "false"
        for x in range(0,10):
            ranknum = ranknum+1
            df = pd.read_csv(MasterSheet) 
            svalue = sortedscore.iloc[x,1]
            GetCPTValue = df.iloc[svalue,0]
            if (GetCPTValue == SameSimSet[NewCodeNum]):
                found = "true"
                break 

        RecNum =0
        for x in range(0,10):
            rownum = rownum+1
            svalue = sortedscore.iloc[x,1]
            Accuracy = sortedscore.iloc[x,2]
            
            Comments = "Present in Autoproposal"

            if (RecNum ==0):
                result = GetMasterCPTCode(svalue,NewCodeSet[NewCodeNum],i,Accuracy*100,ranknum,Comments,rownum)
            else:
                result = GetMasterCPTCode(svalue,NewCodeSet[NewCodeNum],i,Accuracy*100,"",Comments,rownum)
            RecNum =RecNum +1
           
        try:         
            rowNumSameSimFound = resultFrame.loc[(resultFrame.NewCode == NewCodeSet[NewCodeNum]) & (resultFrame.SameSimCode == SameSimSet[NewCodeNum])].index[0]
            resultFrame.loc[rowNumSameSimFound,'Comments'] = "Present Both in MD and Autoproposal"
        except:
            try:
                rownum = rownum+1 
                resultFrame.loc[rownum].NewCode = NewCodeSet[NewCodeNum]
                resultFrame.loc[rownum].SameSimCode = SameSimSet[NewCodeNum]
                resultFrame.loc[rownum].Comments = "Present only by MD"
            except:
                "Nothing"

export_csv = resultFrame.to_csv ('export_dataframe_V3_Grp.csv', index = None, header=True)

resultFrame



10035
10036
31652
31653
31654
33477
37252
37253
39401
39402
43210
47531
47532
47533
47534
47535
47536
47537
47538
47539
47540
47541
47542
47543
47544
49185
50430
50431
50432
50433
50434
50435
50606
50693
50694
50695
50705
50706
54437
54438
61645
61650
61651
64461
64462
64463
65785
69209
72081
72082
72083
72084
73501
73502
73503
73521
73522
73523
73551
73552
74712
74713
77767
77768
77770
77771
77772
78265
78266
80081
81162
81170
81218
81219
81272
81273
81276
81311
81314
81412
81432
81433
81434
81437
81438
81442
81490
81493
81525
81528
81535
81536
81538
81540
81545
81595
88350
90625
92537
92538
93050
96931
96932
96933
96934
96935
96936
99177
99415
99416
0394T
0395T
0396T
0397T
0398T
0399T
0400T
0401T
0402T
0403T
0404T
0405T
0406T
0407T
0408T
0409T
0410T
0411T
0412T
0413T
0414T
0415T
0416T
0417T
0418T
0419T
0420T
0421T
0422T
0423T
0424T
0425T
0426T
0427T
0428T
0429T
0430T
0431T
0432T
0433T
0434T
0435T
0436T
0437T
0438T
0439T
0440T
0441T
0442T
0443T
0444T
0445T
C9137
C9138
C9139
C9461
C947

,NewCode,NewCodeDesc,SameSimCode,CPTDesc,Accuracy,Rank,Comments
0,10035,Placement of soft tissue localization device(s...,10035,Placement of soft tissue localization device(s...,100,10,Present in Autoproposal
1,10035,Placement of soft tissue localization device(s...,19285,Placement of breast localization device(s) (eg...,89.7201,,Present Both in MD and Autoproposal
2,10035,Placement of soft tissue localization device(s...,19283,Placement of breast localization device(s) (eg...,88.737,,Present Both in MD and Autoproposal
3,10035,Placement of soft tissue localization device(s...,10036,Placement of soft tissue localization device(s...,87.3745,,Present in Autoproposal
4,10035,Placement of soft tissue localization device(s...,19287,Placement of breast localization device(s) (eg...,87.1716,,Present Both in MD and Autoproposal
...,...,...,...,...,...,...,...
29995,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29996,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29997,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29998,NaN,NaN,NaN,NaN,NaN,NaN,NaN
